In [427]:
import re

In [10]:
import numpy as np
import pandas as pd

In [443]:
from tqdm import tqdm

In [13]:
import json

In [14]:
from sklearn.metrics.pairwise import cosine_similarity

In [65]:
import random
from phylodm import PhyloDM
import dendropy

In [372]:
co_embedding = pd.read_csv("../../data/social_niche_embedding_100.txt",
                          header=None, sep=" ", low_memory=False, index_col=0)
co_embedding = co_embedding.drop("<unk>")

embed_cos = cosine_similarity(co_embedding)
embed_cos = pd.DataFrame(data=embed_cos, index=co_embedding.index, columns=co_embedding.index)

phy_embedding = pd.read_csv("../../data/Embedding_list/PCA_100.txt",
                          header=None, sep=" ", low_memory=False, index_col=0)
phy_embedding = phy_embedding.loc[co_embedding.index]

embed_cos_phy = cosine_similarity(phy_embedding)
embed_cos_phy = pd.DataFrame(data=embed_cos_phy, index=phy_embedding.index, columns=phy_embedding.index)

### vsearch

In [522]:
from Bio import SeqIO

def get_sequence_lengths(fasta_file):
    seq_lengths = {}
    for record in SeqIO.parse(fasta_file, "fasta"):
        seq_lengths[record.id] = len(record.seq)  # 存储序列ID和长度
    return seq_lengths

# 示例调用
fasta_path = "Data/feces_seq_16S_new.fasta"
length_dict = get_sequence_lengths(fasta_path)

In [523]:
colnames = ["query_id", "refer_id", "identity", "alignment_length", "mismatches", "gap_openings", "q.start",
            "q.end", "s.start", "s.end", "e-value", "bit_score"]

In [524]:
vsearch_out = pd.read_csv("Data/vsearch_blast.out", sep="\t", header=None)
vsearch_out.columns = colnames
vsearch_out.loc[:, "length"] = [length_dict[i] for i in vsearch_out.query_id.values]
vsearch_out.loc[:, "coverage"] = vsearch_out.alignment_length.values / vsearch_out.length.values

In [525]:
vsearch_out = vsearch_out.loc[vsearch_out["q.start"] == 1]
vsearch_out = vsearch_out.loc[vsearch_out["coverage"] >= 1]
vsearch_out.loc[:, "refer"] = [i.split("::")[1].split(":")[0] for i in vsearch_out.refer_id.values]

In [526]:
vsearch_out = vsearch_out.loc[vsearch_out.groupby('refer')['identity'].idxmax()]

In [527]:
vsearch_out.shape

(1125, 15)

In [528]:
def select_sequences_by_ids(input_fasta, output_fasta, selected_ids):
    """
    根据序列名称选择指定的序列并保存到新文件
    
    参数:
        input_fasta (str): 输入 FASTA 文件路径
        output_fasta (str): 输出 FASTA 文件路径
        selected_ids (list): 需要选择的序列名称列表
    """
    
    # 读取输入文件并筛选序列
    selected_sequences = []
    for record in SeqIO.parse(input_fasta, "fasta"):
        if record.id in selected_ids:  # 如果序列名称在指定列表中
            record.id = record.id.split("::")[1].split(":")[0]
            selected_sequences.append(record)
    
    # 将筛选后的序列写入新文件
    SeqIO.write(selected_sequences, output_fasta, "fasta")
    print(f"已保存 {len(selected_sequences)} 条序列到 {output_fasta}")

# 示例调用
target_id = vsearch_out.refer_id.values
input_fasta = "Data/barrnap.fna"  # 输入 FASTA 文件路径
output_fasta = "Data/pick_otu.fasta"  # 输出 FASTA 文件路径
select_sequences_by_ids(input_fasta, output_fasta, target_id) 

已保存 1125 条序列到 /home/dongbiao/word_embedding_microbiome/HGT/16S/pick_otu.fasta


In [529]:
def get_fasta_sequence_names(fasta_file):
    """
    提取单个 FASTA 文件中的所有序列名称
    
    参数:
        fasta_file (str): FASTA 文件路径
    
    返回:
        list: 序列名称列表
    """
    sequence_names = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequence_names.append(record.id)
    return sequence_names

def process_fasta_folder(folder_path):
    """
    处理文件夹中的所有 FASTA 文件，生成字典
    
    参数:
        folder_path (str): 文件夹路径
    
    返回:
        dict: key 是基因组名称（文件名），value 是序列名称列表
    """
    fasta_dict = {}
    
    # 遍历文件夹中的所有文件
    for filename in tqdm(os.listdir(folder_path), desc="Processing"):
        if filename.endswith(".fna"):  # 检查文件扩展名
            file_path = os.path.join(folder_path, filename)
            genome_name = os.path.splitext(filename)[0]  # 去掉扩展名作为基因组名称
            sequence_names = get_fasta_sequence_names(file_path)
            fasta_dict[genome_name] = sequence_names
    
    return fasta_dict

In [530]:
# 示例调用
# folder_path = "/home/dongbiao/word_embedding_microbiome/HGT/high_quality_genome"  
# fasta_dict = process_fasta_folder(folder_path)

Processing: 100%|██████████| 30117/30117 [1:04:08<00:00,  7.83it/s]


In [543]:
# 保存为 JSON 文件
# with open("/home/dongbiao/word_embedding_microbiome/HGT/results/fasta_dict.json", "w") as f:
#     json.dump(fasta_dict, f, indent=4)  # indent 用于美化输出

In [532]:
# 读取 JSON 文件
with open('Data/fasta_dict.json', 'r', encoding='utf-8') as f:
    fasta_dict = json.load(f)  # 返回字典或列表

In [533]:
target_genome = {}
for target in tqdm(vsearch_out.refer.values, desc="Processing"):
    for key, value in fasta_dict.items():
        if target in value:
            target_genome[target] = key
            continue

Processing: 100%|██████████| 1125/1125 [00:28<00:00, 39.93it/s]


In [534]:
vsearch_out.loc[:, "genome_id"] = [target_genome[i] for i in vsearch_out.refer.values]

In [535]:
vsearch_out = vsearch_out.loc[vsearch_out.groupby('genome_id')['identity'].idxmax()]

In [544]:
vsearch_out.shape

(1112, 16)

In [545]:
vsearch_out.head()

,query_id,refer_id,identity,alignment_length,mismatches,gap_openings,q.start,q.end,s.start,s.end,e-value,bit_score,length,coverage,refer,genome_id
60,AJ305238.1.1505,16S_rRNA::DS480346.1:176999-178547(+),99.9,1505,2,0,1,1505,1,1548,-1,0,1505,1.000000,DS480346.1,GCA_000154345.1_genomic
978,ABQR01000074.101.1610,16S_rRNA::DS990270.1:96-1623(+),100.0,1510,0,0,1,1510,1,1527,-1,0,1510,1.000000,DS990270.1,GCA_000155435.1_genomic
133,Y18181.1.1435,16S_rRNA::EQ973341.1:3511-5036(-),99.9,1437,0,2,1,1435,1,1525,-1,0,1435,1.001394,EQ973341.1,GCA_000158655.1_genomic
1900,ACON01000003.712136.713636,16S_rRNA::GG688422.1:1081954-1083464(+),100.0,1501,0,0,1,1501,1,1510,-1,0,1501,1.000000,GG688422.1,GCA_000161975.1_genomic
25,AJ239289.1.1355,16S_rRNA::ADBE01000137.1:300-1836(+),100.0,1355,0,0,1,1355,1,1536,-1,0,1355,1.000000,ADBE01000137.1,GCA_000176735.1_genomic


In [541]:
vsearch_out.to_csv("/home/dongbiao/word_embedding_microbiome/HGT/results/vsearch_res_new.csv", index=None)